In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
FB_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/FB_predictions_SVM.csv", index_col=0)


In [3]:
FB_predictions.head()

Date  Predictions
0  2019-07-31            0
1  2019-06-28            0
2  2019-05-31            1
3  2019-04-30            1
4  2019-03-29            0

In [4]:
FB_predictions.Date = pd.to_datetime(FB_predictions.Date, format="%Y-%m-%d")

In [5]:
FB_predictions['Predictions'] = FB_predictions['Predictions'].shift(-1)


In [6]:
FB_predictions = FB_predictions.dropna()


In [7]:
FB_predictions = FB_predictions[::-1]

In [8]:
FB_predictions.head(10)

Date  Predictions
83 2012-08-31          0.0
82 2012-09-28          1.0
81 2012-10-31          1.0
80 2012-11-30          0.0
79 2012-12-31          1.0
78 2013-01-31          0.0
77 2013-02-28          1.0
76 2013-03-28          0.0
75 2013-04-30          0.0
74 2013-05-31          0.0

In [9]:
FB_monthly = pd.read_csv("/Users/fahad/project_repo/data/final/FB/FB_monthly.csv", index_col =0)

In [10]:
FB_monthly.columns = ['Open','High','Low','Close','Volume']

In [11]:
FB_predictions = FB_predictions.set_index("Date")

In [12]:
FB_monthly.head()

Open      High       Low   Close     Volume
2019-08-15  194.17  198.4700  179.3100  182.59  172260200
2019-07-31  195.21  208.6749  191.9300  194.23  351062500
2019-06-28  175.00  194.5300  160.8316  193.00  426428600
2019-05-31  194.78  196.1770  177.1600  177.47  286821500
2019-04-30  167.83  198.4800  167.2790  193.40  381985300

In [13]:
FB_final_df = pd.merge(FB_predictions, FB_monthly, right_index=True, left_index=True)

In [14]:
FB_final_df.index = pd.to_datetime(FB_final_df.index, format ="%Y-%m-%d")

In [15]:
equity_dict = dict()

In [16]:
class Binary_Classification_1d(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.data.Predictions
        self.set_trailing_sl(4)
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
        
        if self.data.Predictions[-1] == 1.0 and (position.is_long!=True) :
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif self.data.Predictions[-1] == 0.0 and (position.is_short!=True) :
#             print("\n")
#             print (self.data.Prediction[-1])
            
            self.sell()
            
 

In [17]:
bt = Backtest(FB_final_df, Binary_Classification_1d, cash=10000, commission=.02)
bt.run()



Start                          2012-08-31 00:00:00
End                            2019-07-31 00:00:00
Duration                        2525 days 00:00:00
Exposure [%]                               96.2772
Equity Final [$]                            260827
Equity Peak [$]                             260827
Return [%]                                 2508.27
Buy & Hold Return [%]                       975.59
Max. Drawdown [%]                          -19.099
Avg. Drawdown [%]                         -6.97477
Max. Drawdown Duration           397 days 00:00:00
Avg. Drawdown Duration           130 days 00:00:00
# Trades                                        38
Win Rate [%]                               71.0526
Best Trade [%]                             66.5805
Worst Trade [%]                           -8.63671
Avg. Trade [%]                             9.00307
Max. Trade Duration              244 days 00:00:00
Avg. Trade Duration               64 days 00:00:00
Expectancy [%]                 

In [18]:
bt.plot(superimpose=False, plot_volume=False)

In [19]:
df = pd.DataFrame.from_dict([equity_dict])

In [20]:
df = df.T

In [21]:
df.columns = ['Equity']

In [22]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/SVM/FB_SVM_Equity.csv")

In [23]:
df

Equity
2012-09-28   10000.000000
2012-10-31    9373.223927
2012-11-30   12432.509236
2012-12-31   12789.465033
2013-01-31   13760.747102
2013-02-28   15140.879412
2013-03-28   14118.063832
2013-04-30   12710.889231
2013-05-31   14561.979098
2013-06-28   14274.115956
2013-07-31   20555.499373
2013-08-30   23065.728020
2013-09-30   28057.139498
2013-10-31   27235.927145
2013-11-29   28985.576029
2013-12-31   33181.251462
2014-01-31   37990.647660
2014-02-28   41566.880914
2014-03-31   43859.127496
2014-04-30   42388.582212
2014-05-30   44884.530846
2014-06-30   47713.745350
2014-07-31   51514.394407
2014-08-29   48397.934478
2014-09-30   49861.755860
2014-10-31   51067.282654
2014-11-28   51245.674922
2014-12-31   49464.937183
2015-01-30   50829.264209
2015-02-27   48850.666722
...                   ...
2017-02-28   77144.384755
2017-03-31   80849.637409
2017-04-28   85516.775929
2017-05-31   86205.463443
2017-06-30   85932.265090
2017-07-31   96330.877378
2017-08-31   93560.391635
2017-09-29   90678.684648
2017-10-31   95555.708771
2017-11-30   94027.326891
2017-12-29   93645.231421
2018-01-31   99180.308854
2018-02-28  103078.369420
2018-03-29  112719.204193
2018-04-30  121546.576426
2018-05-31  135524.432715
2018-06-29  137319.364716
2018-07-31  148372.177425
2018-08-31  146041.869592
2018-09-28  148726.839611
2018-10-31  159049.886931
2018-11-30  168158.937368
2018-12-31  149380.815122
2019-01-31  189948.036255
2019-02-28  183976.905954
2019-03-29  189948.036255
2019-04-30  220384.847392
2019-05-31  236943.763222
2019-06-28  259174.899471
2019-07-31  260826.635877

[83 rows x 1 columns]